In [1]:
import mido

In [5]:
midi_file = mido.MidiFile("midi/Hallelujah.mid")
midi_length = midi_file.length
print('Song length: {} minutes, {} seconds'.format(int(midi_length / 60), int(midi_length % 60)))

Song length: 4 minutes, 31 seconds


In [11]:
from GeneralMidi import MIDI_GM1_INSTRUMENT_NAMES, MIDI_PERCUSSION_NAMES
from KeyFindingHMM import find_music_key, get_music_key_name, get_root_note_from_music_key, get_scale_from_music_key

tempo = 500000
ticks_per_beat = midi_file.ticks_per_beat
time_signature_numerator = 4
time_signature_denominator = 4
clocks_per_click = 24

count_ticks_in_total = 0
count_ticks_in_beat = 0
count_ticks_in_measure = 0
num_bar = 1
num_beat = 1
current_bar_tick = 0
current_beat_tick = 0
pitch_classes_in_beat = 0
bar_ticks = []

# See: https://www.geeksforgeeks.org/python-find-the-closest-key-in-dictionary/
chords_per_beat = {}

scale_key = 0 # C
scale_type = 0 # Major

pitch_histogram_per_bar = []

full_song = {}

channel_programs = [0] * 16
pitch_histogram = [0] * 12
instruments = set()
for message in mido.midifiles.tracks.merge_tracks(midi_file.tracks):
    total_ticks_in_beat = ticks_per_beat * 4 / time_signature_denominator
    total_ticks_in_measure = ticks_per_beat * time_signature_numerator * 4 / time_signature_denominator
    time_of_measure = mido.midifiles.units.tick2second(total_ticks_in_measure,
                                                       midi_file.ticks_per_beat, tempo)

    notes_on = []
    notes_off = []

    if isinstance(message, mido.Message):
        if message.type == 'note_on':
            if message.channel != 9: # Exclude percussion 
                pitch_histogram[message.note % 12] += 1
                pitch_classes_in_beat |= 1 << (message.note % 12)
                notes_on.append((message.channel, message.note,
                                 channel_programs[message.channel]))
        elif message.type == 'note_off':
            if message.channel != 9: # Exclude percussion 
                pitch_histogram[message.note % 12] -= 1
                notes_off.append((message.channel, message.note))
        elif message.type == 'program_change':
            instruments.add(message.program)
            channel_programs[message.channel] = message.program

        count_ticks_in_total += message.time
        count_ticks_in_measure += message.time
        count_ticks_in_beat += message.time

    elif isinstance(message, mido.MetaMessage):
        if message.type == 'set_tempo':
            tempo = message.tempo
        elif message.type == 'time_signature':
            time_signature_numerator = message.numerator
            time_signature_denominator = message.denominator
            clocks_per_click = message.clocks_per_click
            num_ticks = 0
        elif message.type == 'key_signature':
            #music_key = message.key
            print('Key signature changed to {}'.format(message.key))

    try:
        current_tick_in_song = full_song[count_ticks_in_total]
    except (IndexError, KeyError):
        full_song[count_ticks_in_total] = [None, None, None, [], []]
        current_tick_in_song = full_song[count_ticks_in_total]
    current_tick_in_song[3] += notes_on
    current_tick_in_song[4] += notes_off

    while count_ticks_in_beat >= total_ticks_in_beat:
        num_beat += 1
        count_ticks_in_beat -= total_ticks_in_beat
        chords_per_beat[current_beat_tick] = pitch_classes_in_beat
        full_song[current_beat_tick][2] = pitch_classes_in_beat
        print(f"beat@{count_ticks_in_total}: {num_beat}:{current_beat_tick} -> " +
               f"{pitch_classes_in_beat:#06x} = {pitch_classes_in_beat:>012b}")
        current_beat_tick = count_ticks_in_total
        pitch_classes_in_beat = sum([1 << (n % 12) if pitch_histogram[n] > 0 else 0 for n in range(12)])

    while count_ticks_in_measure >= total_ticks_in_measure:
        h = sum([1 << (n % 12) if pitch_histogram[n] > 0 else 0 for n in range(12)])
        pitch_histogram_per_bar.append(h)
        full_song[current_bar_tick][1] = h
        print(f"Bar #{num_bar}: {pitch_histogram} ({time_of_measure:1f} s) -> {h:03x} ~ {h:012b}")
        bar_ticks.append(current_bar_tick)
        num_bar += 1
        current_bar_tick = count_ticks_in_total
        count_ticks_in_measure -= total_ticks_in_measure

print(f"end@{count_ticks_in_total}: {num_beat}:{current_beat_tick} -> " +
       f"{pitch_classes_in_beat:#06x} = {pitch_classes_in_beat:>012b}")
chords_per_beat[current_beat_tick] = pitch_classes_in_beat

if count_ticks_in_measure:
    bar_ticks.append(current_bar_tick)

music_key_per_bar = find_music_key(pitch_histogram_per_bar)
print(['{:03x}={}'.format(v, get_music_key_name(s)) for v, s in zip(pitch_histogram_per_bar, music_key_per_bar)])

for i, (v, s) in enumerate(zip(pitch_histogram_per_bar, music_key_per_bar)):
    full_song[bar_ticks[i]][0] = s
    full_song[bar_ticks[i]][1] = v

print(f"end: {pitch_histogram}")
          
print([MIDI_GM1_INSTRUMENT_NAMES[i + 1] for i in instruments])

#print(full_song)


Key signature changed to C
beat@60: 2:0 -> 0x0091 = 000010010001
beat@120: 3:60 -> 0x0091 = 000010010001
beat@180: 4:120 -> 0x0091 = 000010010001
beat@240: 5:180 -> 0x0091 = 000010010001
beat@300: 6:240 -> 0x0091 = 000010010001
beat@360: 7:300 -> 0x0891 = 100010010001
Bar #1: [2, 0, 0, 0, 2, 0, 0, 3, 0, 0, 0, 2] (1.935483 s) -> 891 ~ 100010010001
beat@420: 8:360 -> 0x0a91 = 101010010001
beat@480: 9:420 -> 0x0211 = 001000010001
beat@540: 10:480 -> 0x0211 = 001000010001
beat@600: 11:540 -> 0x0211 = 001000010001
beat@660: 12:600 -> 0x0211 = 001000010001
beat@720: 13:660 -> 0x0a11 = 101000010001
Bar #2: [1, 0, 0, 0, 2, 0, 0, 0, 0, 5, 0, 2] (1.935483 s) -> a11 ~ 101000010001
beat@780: 14:720 -> 0x0a91 = 101010010001
beat@840: 15:780 -> 0x0091 = 000010010001
beat@900: 16:840 -> 0x0091 = 000010010001
beat@960: 17:900 -> 0x0091 = 000010010001
beat@1020: 18:960 -> 0x0091 = 000010010001
beat@1080: 19:1020 -> 0x0891 = 100010010001
Bar #3: [2, 0, 0, 0, 2, 0, 0, 3, 0, 0, 0, 2] (1.935483 s) -> 891 ~

['891=C:Maj', 'a11=C:Maj', '891=C:Maj', 'a11=C:Maj', '891=C:Maj', 'a11=C:Maj', '891=C:Maj', 'a11=C:Maj', '221=C:Maj', '884=C:Maj', '891=C:Maj', '884=C:Maj', '891=C:Maj', '884=C:Maj', 'a11=C:Maj', '221=C:Maj', '884=C:Maj', '890=C:Maj', '211=C:Maj', '211=C:Maj', '221=C:Maj', '221=C:Maj', 'a11=C:Maj', 'a11=C:Maj', '221=C:Maj', '221=C:Maj', '891=C:Maj', '884=C:Maj', '891=C:Maj', '894=C:Maj', 'a11=C:Maj', '891=C:Maj', 'a11=C:Maj', '891=C:Maj', 'a11=C:Maj', '221=C:Maj', '884=C:Maj', '891=C:Maj', '884=C:Maj', '891=C:Maj', '884=C:Maj', 'a11=C:Maj', '221=C:Maj', '884=C:Maj', '890=C:Maj', '211=C:Maj', '211=C:Maj', '221=C:Maj', '221=C:Maj', 'a11=C:Maj', 'a11=C:Maj', '221=C:Maj', '221=C:Maj', '891=C:Maj', '884=C:Maj', '891=C:Maj', '894=C:Maj', 'a11=C:Maj', '891=C:Maj', 'a11=C:Maj', '891=C:Maj', 'a11=C:Maj', '221=C:Maj', '884=C:Maj', '891=C:Maj', '884=C:Maj', '891=C:Maj', '884=C:Maj', 'a11=C:Maj', '221=C:Maj', '884=C:Maj', '890=C:Maj', '211=C:Maj', '211=C:Maj', '221=C:Maj', '221=C:Maj', 'a11=C:Maj'